In [ ]:
#=======================================================================================================
# This python notebook will retrain incetion V3 Model, save the weights and make predictions
#=======================================================================================================

# The directory structure is as follows:
#  root
#  |
# data
#  |
#  flowers
#  |
#  - train
#      - roses
#      - daisy
#  - valid
#      - roses
#      - daisy    

from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input

# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir ='data/flowers/sample/train' #contains two classes cats and dogs
validation_data_dir ='data/flowers/sample/valid' #contains two classes cats and dogs



# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)

# and a logistic layer -- let's say we have 2 classes
predictions = Dense(2, activation='softmax')(x)

# this is the model we will train
model = Model(input=base_model.input, output=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
#model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])


print('\n')
print('===========================')
print('Model Compilation Complete')
print('===========================')

In [ ]:
# Summarize the Model Parameters
model.summary()

In [ ]:
# Now specify the Directories for Training and Validation

nb_train_samples = 697 #1274
nb_validation_samples = 100 #1274

# Parameters for Training
nb_epoch = 2 # The Number of Epochs to Train over
b_size = 50 # The Batch Size
samp_per_epoch = 500 # The 

train_data_dir ='data/flowers/train' #contains two classes cats and dogs
validation_data_dir ='data/flowers/valid' #contains two classes cats and dogs

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
        rescale=1./255)#,
 #       shear_range=0.2,
 #       zoom_range=0.2,
 #       horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=b_size,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=b_size,
    class_mode='categorical'
)

model.fit_generator(
    train_generator,
    nb_epoch=nb_epoch,
    samples_per_epoch=samp_per_epoch, #128
    validation_data=validation_generator,
    nb_val_samples= nb_validation_samples) #1020

print('\n')
print('====================================================')
print('Training Base Model Complete: \n')
print('====================================================')

In [ ]:
# Now that top layers of the  original model have been trained we will test the predictions

from keras.preprocessing import image
from keras.applications.inception_v3 import preprocess_input
import numpy as np

import shutil
from shutil import copyfile, move
import os
from os import listdir
from os.path import isfile, join


mypath='./daisy'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
#print(onlyfiles)

avg1=0.0
avg2=0.0

for file in onlyfiles:
     # Let us use the inception v3 Model
    img_path = mypath+'/'+file
    #print(img_path)
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict(x)
    class1 = preds[0][0]
    class2 = preds[0][1]
    avg1 = avg1+class1
    avg2 = avg2+class2
    
print('Daisy Class 1: '+str(avg1/float(len(onlyfiles))))
print('Daisy Class 2: '+str(avg2/float(len(onlyfiles))))


mypath='./roses'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
#print(onlyfiles)

avg1=0.0
avg2=0.0

for file in onlyfiles:
     # Let us use the inception v3 Model
    img_path = mypath+'/'+file
    #print(img_path)
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict(x)
    class1 = preds[0][0]
    class2 = preds[0][1]
    avg1 = avg1+class1
    avg2 = avg2+class2

print('\n')
print('===============================================')    
print('Roses Class 1: '+str(avg1/float(len(onlyfiles))))
print('Roses Class 2: '+str(avg2/float(len(onlyfiles))))

In [ ]:
#===================================================================================
# Now that some of the testing was satisfactory, we can save this InceptionV3 Model
#===================================================================================

model.save("inceptionV3_retrain_top")
##################################

print('\n')
print('====================================================')
print('Saving The Model is Complete: \n')
print('====================================================')
